# Preparing the test set

#### Importing the data

In [7]:

# @hidden_cell
credentials_6 = {
  'auth_url':'https://identity.open.softlayer.com',
  'project':'object_storage_4ce2b21e_bf48_4e98_948d_2c0b81b8b84f',
  'project_id':'8160254a979c4ad99607776ef3ddb00b',
  'region':'dallas',
  'user_id':'5d948dc7699c4f8f8625676ee15ecb9b',
  'domain_id':'8d5c5723b93e4bdf8c5be53de58a2395',
  'domain_name':'1145017',
  'username':'admin_c24f02ccead8f3d4baea656ecbccd8e6e38d474d',
  'password':"""D!~GY/p7Z]b##1oV""",
  'container':'seminaire_data',
  'tenantId':'undefined',
  'filename':'Transformed_features.csv'
}


In [8]:

# @hidden_cell
from io import StringIO
import requests
import json
import pandas as pd

# This function accesses a file in your Object Storage. The definition contains your credentials.
# You might want to remove those credentials before you share your notebook.
def get_object_storage_file_with_credentials_4ce2b21ebf484e98948d2c0b81b8b84f(container, filename):
    """This functions returns a StringIO object containing
    the file content from Bluemix Object Storage V3."""

    url1 = ''.join(['https://identity.open.softlayer.com', '/v3/auth/tokens'])
    data = {'auth': {'identity': {'methods': ['password'],
            'password': {'user': {'name': 'admin_c24f02ccead8f3d4baea656ecbccd8e6e38d474d','domain': {'id': '8d5c5723b93e4bdf8c5be53de58a2395'},
            'password': 'D!~GY/p7Z]b##1oV'}}}}}
    headers1 = {'Content-Type': 'application/json'}
    resp1 = requests.post(url=url1, data=json.dumps(data), headers=headers1)
    resp1_body = resp1.json()
    for e1 in resp1_body['token']['catalog']:
        if(e1['type']=='object-store'):
            for e2 in e1['endpoints']:
                        if(e2['interface']=='public'and e2['region']=='dallas'):
                            url2 = ''.join([e2['url'],'/', container, '/', filename])
    s_subject_token = resp1.headers['x-subject-token']
    headers2 = {'X-Auth-Token': s_subject_token, 'accept': 'application/json'}
    resp2 = requests.get(url=url2, headers=headers2)
    return StringIO(resp2.text)

data_test = pd.read_csv(get_object_storage_file_with_credentials_4ce2b21ebf484e98948d2c0b81b8b84f('seminaire_data', 'Test_set_X.csv'))
data_test.head()
data_train = pd.read_csv(get_object_storage_file_with_credentials_4ce2b21ebf484e98948d2c0b81b8b84f('seminaire_data', 'Training_setX.csv'))
data_train.head()
data_transformed = pd.read_csv(get_object_storage_file_with_credentials_4ce2b21ebf484e98948d2c0b81b8b84f('seminaire_data', 'Transformed_features.csv'))
data_transformed.head()


,Unnamed: 0,installer_f,installer_fnr,extraction_type_f,extraction_type_fnr,source_f,source_fnr,waterpoint_type_group_f,waterpoint_type_group_fnr,management_f,...,scheme_management_fnr,lga_f,lga_fnr,basin_f,basin_fnr,amount_tsh,permit,age_pump,public_meeting,target
0,0,0.606383,0.074468,0.599253,0.100859,0.622290,0.074966,0.576491,0.083986,0.504234,...,0.063436,0.774823,0.019504,0.653687,0.049164,6000,0,36,1,0
1,1,0.846154,NaN,0.599253,0.100859,0.603922,0.136819,0.576491,0.083986,0.599540,...,0.028721,0.592179,0.037709,0.497658,0.096507,0,1,47,-1,0
2,2,0.654412,0.132353,0.599253,0.100859,0.385671,0.036585,0.576491,0.083986,0.504234,...,0.063436,0.581169,0.016234,0.600895,0.053356,25,1,46,1,0
3,3,0.536036,0.040541,0.551217,0.047649,0.489571,0.044334,0.576491,0.083986,0.504234,...,0.063436,0.265823,NaN,0.371689,0.072557,0,1,23,1,2
4,4,0.555556,0.044444,0.599253,0.100859,0.603922,0.136819,0.576491,0.083986,0.598341,...,NaN,0.530480,0.058366,0.497658,0.096507,0,1,-1,1,0


#### Repeating the same approach as for the training set

In [9]:
# Transforming the features as described in the "exploring the data" part.
list_features_to_transform = ["installer","extraction_type","source","waterpoint_type_group","management","payment","water_quality","quantity","region","scheme_management","lga", "basin"]

In [10]:
data_new = pd.DataFrame()
dim = data_transformed.shape[0]

for feature in list_features_to_transform :
    # f for functional
    mean = data_transformed.mean()
    data_transformed.loc[dim] = list(mean)
    print(feature)
    l_indexes = [list(data_train[feature]).index(value) if value in  data_train[feature].tolist() else dim for value in data_test[feature].tolist()]
    data_new[feature + "_f"] = list(data_transformed[feature + "_f"].iloc[l_indexes])
    data_new[feature + "_fnr"] = list(data_transformed[feature + "_fnr"].iloc[l_indexes])

installer
extraction_type
source
waterpoint_type_group
management
payment
water_quality
quantity
region
scheme_management
lga
basin


In [11]:
# Transforming the other features
data_new["amount_tsh"] = data_test["amount_tsh"]

In [12]:
# filling the Nan by -1
data_new["permit"] = data_test["permit"].fillna(-1).astype(int)

In [13]:
# replacing the date
data_new["age_pump"] = data_train["construction_year"].apply(lambda x : x - 1963 if x != 0 else -1)

In [14]:
# filling the Nan by -1
data_new["public_meeting"] = data_train["public_meeting"].fillna(-1).astype(int)

In [15]:
data_new.head()

,installer_f,installer_fnr,extraction_type_f,extraction_type_fnr,source_f,source_fnr,waterpoint_type_group_f,waterpoint_type_group_fnr,management_f,management_fnr,...,scheme_management_f,scheme_management_fnr,lga_f,lga_fnr,basin_f,basin_fnr,amount_tsh,permit,age_pump,public_meeting
0,0.741573,0.078652,0.160031,0.032037,0.603922,0.136819,0.131661,0.045925,0.576923,0.119344,...,0.575000,0.120238,0.643098,0.033670,0.575723,0.071548,0,1,36,1
1,0.542064,0.093208,0.599253,0.100859,0.622290,0.074966,0.576491,0.083986,0.504234,0.068902,...,0.515315,0.063436,0.698882,0.038339,0.600895,0.053356,0,1,47,-1
2,NaN,NaN,0.160031,0.032037,0.603922,0.136819,0.131661,0.045925,0.504234,0.068902,...,0.515315,0.063436,0.440201,0.062312,0.575723,0.071548,0,-1,46,1
3,0.060606,NaN,0.160031,0.032037,0.494769,0.056883,0.131661,0.045925,0.504234,0.068902,...,0.515315,0.063436,0.253247,0.019481,0.371689,0.072557,0,1,23,1
4,0.812500,0.062500,0.599253,0.100859,0.622290,0.074966,0.576491,0.083986,0.739857,0.090351,...,0.747089,0.040393,0.526667,0.076000,0.371689,0.072557,500,1,-1,1


#### Writting the data

In [16]:
data_new.to_csv("Transformed_features_submission.csv")

In [17]:
# writting the data

In [18]:
from io import BytesIO  
import requests  
import json  
import pandas as pd

def put_file(credentials, local_file_name):  
    """This functions returns a StringIO object containing
    the file content from Bluemix Object Storage V3."""
    f = open(local_file_name,'r')
    my_data = f.read()
    url1 = ''.join(['https://identity.open.softlayer.com', '/v3/auth/tokens'])
    data = {'auth': {'identity': {'methods': ['password'],
            'password': {'user': {'name': credentials['username'],'domain': {'id': credentials['domain_id']},
            'password': credentials['password']}}}}}
    headers1 = {'Content-Type': 'application/json'}
    resp1 = requests.post(url=url1, data=json.dumps(data), headers=headers1)
    resp1_body = resp1.json()
    for e1 in resp1_body['token']['catalog']:
        if(e1['type']=='object-store'):
            for e2 in e1['endpoints']:
                        if(e2['interface']=='public'and e2['region']=='dallas'):
                            url2 = ''.join([e2['url'],'/', credentials['container'], '/', local_file_name])
    s_subject_token = resp1.headers['x-subject-token']
    headers2 = {'X-Auth-Token': s_subject_token, 'accept': 'application/json'}
    resp2 = requests.put(url=url2, headers=headers2, data = my_data )
    print resp2

In [19]:
put_file(credentials_6, "Transformed_features_submission.csv")

<Response [201]>
